In [1]:
import logging
from pprint import pprint
from hashlib import md5
from time import time
import requests
import json

Задание 1   

Посмотреть документацию к API гитхаба, разобраться как вывести список репозиториев для конкретного пользователя, сохранить JSON-вывод в файле *.json

#### Подготовка класса

In [2]:
config = {
    "main-url": "https://api.github.com/"
}


class GitHubUrl:
    url_template = {
        "repos_url": "users/{}/repos" 
    }
    
    def __init__(self, config: dict):
        self.config = config
    
    @property
    def base(self):
        return self.config["main-url"]

    def repos(self, user):
        url = self.base + \
              self.url_template["repos_url"]\
                  .format(user)
        return self._do_request(url)

    def _do_request(self, url, **kwargs) -> requests.Response:
        return requests.get(url, **kwargs)
    
    def write(self, f_name):
        with open(f"/tmp/{f_name}", "w") as f:
            f.write(json.dumps(repos))
    

#### Сбор имен репозиториев по гитхаб для пользователя malinich

In [3]:
service = GitHubUrl(config) 
repos = service\
    .repos("malinich")\
    .json()

names = [
    repo["name"] for repo in repos
]
pprint(names)

['awesome',
 'awesomeWM',
 'booknotes',
 'codejam',
 'ep2016',
 'Flask-Admin-Dashboard',
 'GDG-Exercise-1',
 'GDG-Exercise-2',
 'GDG-FINAL_PRJ',
 'geekbrains',
 'geekbrains_2_python_for_pandas',
 'Geekbrains_DB_0607',
 'greenback',
 'icl-back',
 'ICL-FRONT',
 'inmem_vms',
 'Junior_dev',
 'JWMeetMe',
 'marvel',
 'MarvelSystem',
 'MarvelSystemV2',
 'pdfImageParser',
 'pika',
 'reportas',
 'retrowatch',
 'settings',
 'small-coerce',
 'Tools',
 'Virtualization-Documentation',
 'wli']


Сохраняем в файлик

In [4]:
service.write("github2.json")

Задание 2  

Изучить список открытых API. Найти среди них любое, которое требует авторизацию (любого типа). Выполнить запросы к нему, пройдя авторизацию.  Результат ответа от сервера записать в файл 



In [5]:
# ключи затерты хххх для безопасности, так как это уедет в репозиторий гитхаб
config = {
    "main_url": "http://gateway.marvel.com",
    "public": "xxxx65b38cdcd75799f1f053xxxxxxxx",
    "private": "xxxx91fef4d4316e3df0009f00axxxxxxxxxxxxx"
} 

In [11]:
class MarvelApi:
    url_template = {
        "comics": "/v1/public/comics" 
    }
        
    def __init__(self, config):
        self.config = config
    
    @property
    def base(self):
        return self.config['main_url']

    @property
    def _private(self):
        return self.config['private']
    @property
    def _public(self):
        return self.config['public']

    def generate_auth_hash(self, ts):
        auth_str:str = f"{ts}" \
                       f"{self._private}" \
                       f"{self._public}"
        return md5(auth_str.encode("utf-8")).hexdigest()

    def comics(self):
        url = self.base + \
              self.url_template["comics"]
        return self._do_request(url, {})
    
    def _do_request(self, url, params: dict, **kwargs) -> requests.Response:
        ts=int(time())
        auth_hash = self.generate_auth_hash(ts) 
        auth_hash_data = {
            "ts": ts,
            "apikey": self._public,
            "hash": auth_hash
        }
        params.update(auth_hash_data)
        return requests.get(url, params=params, **kwargs)

    def write(self, f_name):
        with open(f"/tmp/{f_name}", "w") as f:
            f.write(json.dumps(repos))

In [12]:
marver_service = MarvelApi(config)

In [13]:
comics = marver_service.comics().json()

In [15]:
pprint([comic["title"] for comic in comics['data']['results']])

['Spider-Man/Deadpool (2016) #55',
 'Ms. Marvel (2015) #45',
 'X-Men: Days of Future Past (Trade Paperback)',
 'Ant-Man (2003) #1',
 'Official Handbook of the Marvel Universe (2004) #9 (THE WOMEN OF MARVEL)',
 'Official Handbook of the Marvel Universe (2004) #13 (TEAMS)',
 'Ant-Man (2003) #4',
 'Marvel Milestones (2005) #22',
 'Gun Theory (2003) #3',
 'Official Handbook of the Marvel Universe (2004) #14 (FANTASTIC FOUR)',
 'Official Handbook of the Marvel Universe (2004) #12 (SPIDER-MAN)',
 'Ultimate X-Men Vol. 5: Ultimate War (Trade Paperback)',
 'Ant-Man (2003) #2',
 'Amazing Spider-Man 500 Covers Slipcase - Book II (Trade Paperback)',
 'Sentry, the (Trade Paperback)',
 'Ant-Man (2003) #3',
 'Startling Stories: The Incorrigible Hulk (2004) #1',
 'Official Handbook of the Marvel Universe (2004) #11 (X-MEN - AGE OF '
 'APOCALYPSE)',
 'Marvel Age Spider-Man Vol. 2: Everyday Hero (Digest)',
 'Gun Theory (2003) #4']


In [16]:
marver_service.write("comics.json")